# Changes in the composition of Advanced Placement test-takers over time

For a recent project, our team wanted to explore how scores on Advanced Placement tests, and the composition of test-takers, has evolved over time, and whether these changed during the COVID-19 pandemic. Longitudinal AP data, however, must be manually downloaded for each school year. 

<img src='imgs/apdata.png' width="400" height="400">

To circumvent this and to avoid the tedium of manually downloading more than 20 files, I wrote a Stata script that shells to the cURL program to download the .html page, parse it to extract the URLs for each file, and restructure the file to a single-record layout, by student testing group and school year. All data were obtained from the annual [National Summary reports](https://secure-media.collegeboard.org/digitalServices/misc/ap/national-summary-2019.xlsx). I've migrated this project over to a Jupyter notebook using Stata 17's new native IPython integration. 

In [1]:
##Stata setup
import stata_setup
stata_setup.config("/Applications/Stata", "mp")


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706316772
  Licensed to: Marshall Garland
               Gibson Consulting Group

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.

Running /Users/mgarland/Library/Application Support/Stata/profile.do ...
Profile is here:/Users/mgarland/Library/Application S

## Setting up the local project folder

In [2]:
%%stata -qui
discard
macro drop _all
set trace off
preamble, rootf(/Users/mgarland/Documents/) clientf(GitHub) base(apdata) ///
	sub(syntax logs imgs final raw converted csv website) ///
	mkdir log logpath(logs) logname(1_exploring_ap_data) ///
	scheme(plotplainblind_mwg) clear
graph set window fontface default

/*
## Directory structure for the AP website
Main path:
https://research.collegeboard.org/programs/ap/data/archived/
Download path:
https://research.collegeboard.org/programs/ap/data/archived/ap-2016 <--note that in some years, the last page is indexed with just the year. 
File location: https://secure-media.collegeboard.org/digitalServices/misc/ap/national-summary-2016.xls
*/

//Additional globals
global url="https://research.collegeboard.org/programs/ap/data/archived/"
global url2="https://research.collegeboard.org/programs/ap/data/participation/ap-2020"
if c(os)=="MacOSX" | c(os)=="Unix" {
	global shell "shell"
	global curl "${shell} curl"
	global wget "${shell} wget"
}

global date_time `"`=subinstr("`:di %tc (clock("`c(current_date)' `c(current_time)'", "DMYhms"))'", ":", "_", .)'"'
global date `"`=subinstr("`:di %td (date("`c(current_date)'", "DMY"))'", ":", "_", .)'"'
global outputfile "${csv}ap_national_report_data.xlsx"
global step1=0 //1 to repeat the html download (step 1)
global step2=0 //1 to clean the files harvested from step1
global step3=0 //1 to re-download the .xlsx files
global step4=0 //consolidate annual xlsx files into a single workbook
global step5=0 //cleanup the final Stata .dta file
global fyear=2002 //first year of data
global lyear=2020 //last year of data
global sub="ap-"
global txtfile "${website}/ap_page_data"

## Step 1: Downloading the separate .html pages for each test year and saving as .txt files

In [3]:
%%stata -qui
//Downloading
//Shelling to CURL to download html page and parse for all available downloads.
cd "${website}"
if ${step1}==1 {
	quietly rmfiles, folder("${website}") match("*.txt")	
	local cnt=0
	numlist "${fyear}/${lyear}"
	foreach x in `r(numlist)' {
		local url
		if inrange(`x', 2014,2019) local url="${url}${sub}`x'"
		if inrange(`x', 2002,2013) local url="${url}`x'"
		if `x'==2020 local url="${url2}"
		di in red `"`url'"'
		${curl} -s -S -o "${txtfile}_`x'.txt" `url'
		local ++cnt
	}
	
	quietly dirlist *.txt
	assert `cnt'==`r(nfiles)'
}

## Step 2: Parsing the annual .html files to obtain the download URLs

In [4]:
%%stata -qui
if ${step2}==1 {
    clear
    gen strL file=""
    gen year=""
    numlist "${fyear}/${lyear}"
    local counter=0
    foreach x in `r(numlist)' {
        tempname txt
        file open `txt' using `"${txtfile}_`x'.txt"', read
        file read `txt' line
        local linenum=0
        local stop=0
        while `stop'==0 {
            file read `txt' line
            *di in red `"Stop: `stop': Line number `linenum', `r(status)', `r(eof)'"'
            if strmatch(upper(`"`macval(line)'"'), "*NATIONA*.XL*")==1 | strmatch(upper(`"`macval(line)'"'), "*TIONAL*.XLS*")==1  {
                local counter=`counter'+1		
                display %4.0f `linenum' %4.0f `counter' _asis `"  `macval(line)'"'
                local new=_N+1
                set obs `new'
                replace file=`"`macval(line)'"' in `counter'
                replace year="`x'" in `counter'
            }
            local ++linenum
            if strmatch(trim(`"`macval(line)'"'), "</html>")==1 local stop=1		
        }
        file close `txt'
    }

    save "${raw}/ap_website_html_data.dta", replace   
}

## Step 3: Cleaning up the .html results and downloading the annual AP data files

In [5]:
%%stata -qui
if ${step3}==1 {
    use "${raw}/ap_website_html_data.dta", clear             
    cap drop _*
    //Cleaning up the rows
    moss file, match(`"(http[s]?://[_/a-zA-Z0-9\.\-]+.xls[x]?)"') regex
    cap drop url
    gen url=""
    quietly foreach var of varlist _match* {
        replace url=`var' if strmatch(upper(`var'), "*NATIO*") & mi(url)
    }

    assert url!=""
    keep url year
    replace url=trim(url)

    cd "${csv}"
    cap rm "${outputfile}"
    drop if url==""
    local cnt=1
    local putcnt=2
    quietly forvalues i=1/`=_N' {
        di in red "Now, `=url[`i']', year `=year[`i']'"
        local newfn
        local table
        local sheetname	
        local newfn "ap_national_`=year[`i']'.xls"
        local year "`=year[`i']'"
        ${curl} `=url[`i']' -L --output "${csv}`newfn'"
        if `i'==`=_N' {
            quietly dirlist "*xls"
            assert `r(nfiles)'==`=_N'
        }
    }
}

## Step 4: Consolidating the .xlsx files into a single Excel workbook

In [6]:
%%stata -qui
if ${step4}==1{
    cd "${csv}"
    cap rm "${outputfile}"
    quietly fs *xls
    foreach f in `r(files)' {
        di in red "Now, `f'"
        quietly import excel using "${csv}/`f'", describe
        local sheets=`r(N_worksheet)'
        forvalues i=1/`sheets' {
            if `"`=trim("`=upper("`r(worksheet_`i')'")'")'"'=="ALL" {
                tokenize "`f'", parse("_")
                local filename=subinstr("`5'", ".xls", "", .)
                preserve
                    import excel using "${csv}/`f'", sheet("`r(worksheet_`i')'") clear
                    drop in 1/4
                    export excel using "${outputfile}", sheet("`filename'", replace)
                restore
            }
        }
    }
}

## Step 5: Cleaning!

In [12]:
    %%stata -qui
if ${step5}==1 {
    //Whoo, buddy, this file is a doozy! There are padded columns and rows, and it needs to be reshaped into a single line, per year, for each group. Here goes!
    local first=2002
    local last=2020
    numlist "`first'/`last'"
    foreach i in `r(numlist)' {
    di in red "Now, `i'"	
    import excel using "${outputfile}", sheet("`i'") clear
    missings dropobs *, force
    *Removing extraneous rows
    tempvar aa
    egen `aa'=rowmiss(*)
    drop if inrange(`aa', 10, 10000)
    *Now, extraneous columns
    tempvar aa
    gen `aa'=0
    replace `aa'=1 if strmatch(A, "*MEAN*")
    quietly sum `aa'
    if `r(max)'==0 {
        drop A
    }

    rename * var_#, renumber
    *Now, filling in empty race/ethnicity rows. 
    keep var_1 var_2 var_3
    multitrim var_1 var_2 var_3
    carryforward var_1, replace
    *Keeping the total row

    rename var_1 group
    rename var_2 score
    rename var_3 n_
    drop if strmatch(group, "*MEAN*")
    replace group=upper(trim(group))
    replace group="NATIONAL TOTAL" if group=="TOTAL"	
    drop if mi(group)
    destring n_, replace ignore(`"*"')
    quietly compress	
    quietly reshape wide n_, i(group) j(score) string
    gen year=`i'
    quietly compress
    cap drop __*
    save "${converted}ap_race_ethnicity_`i'.dta", replace
    clear
    }
    //Begin final cleanup!
    cd "${converted}"
    quietly fs ap_race*.dta
    append using `r(files)', force
    *Consolidating categories
    cap drop group2
    gen group2=.
    replace group2=0 if regexm(group, `"(AMER IND./ALASK|AMERICAN)"') //am ind/alask nat
    replace group2=1 if regexm(group, `"(ASIAN|HAWAIIAN)"') //asian/pac island
    replace group2=2 if regexm(group, `"(HISPANIC|LATINO|MEXICAN|PUERTO)"') //hispanic/latino
    replace group2=3 if regexm(group, `"(BLACK)"') //black
    replace group2=4 if regexm(group, `"(WHITE)"') //white
    replace group2=5 if regexm(group, `"(STATED|TWO|RESPONSE)"') //other (including two or more, not stated, and other
    replace group2=6 if regexm(group, `"(NATIONAL)"')
    replace group2=5 if group=="OTHER"
    assert group2!=.
    drop group
    rename group2 group
    collapse (sum) n_*, by(group year)
    renvars n_*, postfix(_)
    label define group2 0 "Am. Ind./Alask. Nat." 1 "Asian" 2 "Hispanic" 3 "Black" 4 ///
    "White" 5 "Other" 6 "Total", modify
    label values group group2
    tempfile yyy
    label save group2 using `yyy', replace
    reshape wide n_*, i(year) j(group)
    quietly do `yyy'
    quietly label list group2
    forvalues i=`r(min)'/`r(max)' {
	foreach var of varlist n_*_`i' {
		tokenize `var', parse("_")
		local score
		if "`3'"!="T" {
			local score=`3'
		}
		else {
			local score="Total test-takers"
		}
		label variable `var' `"`:label group2 `i'': `score' (count)"'
	}
                            
	cap drop p_T_`i'
	gen p_T_`i'=(n_T_`i'/n_T_6)*100
	label variable p_T_`i' `"`:label group2 `i'' (percent)"'
	
	*Now, percentage with a qualifying score.
	cap drop p_qual_score_`i'
	gen p_qual_score_`i'=((n_3_`i'+n_4_`i'+n_5_`i')/n_T_`i')*100
	label variable p_qual_score_`i' `"Percentage with a qualifying score: `:label group2 `i''"'
    }
    label variable year "Testing year"
    compress
    cap drop __*
    save "${final}ap_test_data_`first'_`last'.dta", replace
    export excel using "${final}ap_test_data_`first'_`last'.xlsx", firstrow(varlabels) replace
}                    

## Exploring how AP test-taking has changed since 2002
The graphing code below is adapated from examples posted on the Stata [TechTips](https://www.techtips.surveydesign.com.au/post/stacked-area-plot
) website. 

In [13]:
%%stata -qui -nogr
use "${final}ap_test_data_${fyear}_${lyear}.dta", clear
local graph

// create the graph
cap drop zero
cap drop l?
gen zero = 0
gen l1 = (p_T_0)
gen l2 = (p_T_0 + p_T_1)
gen l3 = (p_T_0 + p_T_1+p_T_2)
gen l4 = (p_T_0 + p_T_1+p_T_2+p_T_3)
gen l5 = (p_T_0 + p_T_1+p_T_2+p_T_3+p_T_4)
gen l6 = (p_T_0 + p_T_1+p_T_2+p_T_3+p_T_4+p_T_5)

sort year
quietly sum year
global min=`r(min)'
global max=`r(max)'
// collect the labels for the second y-axis
quietly sum p_T_0 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_0 if year==${max}
local l12: di %5.1f `r(max)'
local mid = l1[_N]/2
local yaxis `"`mid' "Am. Ind.:`l11'%-`l12'%""'

quietly sum p_T_1 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_1 if year==${max}
local l12: di %5.1f `r(max)'
local mid = (l2[_N]-l1[_N])/2 + l1[_N]
local yaxis `"`yaxis' `mid' "Asian:`l11'%-`l12'%""'

quietly sum p_T_2 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_2 if year==${max}
local l12: di %5.1f `r(max)'
local mid = (l3[_N]-l2[_N])/2 + l2[_N]
local yaxis `"`yaxis' `mid' "Hispanic:`l11'%-`l12'%""'

quietly sum p_T_3 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_3 if year==${max}
local l12: di %5.1f `r(max)'
local mid = (l4[_N]-l3[_N])/2 + l3[_N]
local yaxis `"`yaxis' `mid' "Black:`l11'%-`l12'%""'

quietly sum p_T_4 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_4 if year==${max}
local l12: di %5.1f `r(max)'
local mid = (l5[_N]-l4[_N])/2 + l4[_N]
local yaxis `"`yaxis' `mid' "White:`l11'%-`l12'%""'

quietly sum p_T_5 if year==${min}
local l11: di %5.1f `r(max)'
quietly sum p_T_5 if year==${max}
local l12: di %5.1f `r(max)'
local mid = (100-l5[_N])/2 + l5[_N]
local yaxis `"`yaxis' `mid' "Other:`l11'%-`l12'%""'

local lcolors="lwidth(none)"
colorpalette viridis, nograph
twoway rarea zero l1 year, color("`r(p1)'") `lcolors' yaxis(1) ||  ///
rarea l1 l2 year, yaxis(2)  `lcolors' color("`r(p3)'")  ||  ///
rarea l2 l3 year, `lcolors' color("`r(p6)'")  ||            ///
rarea l3 l4 year, `lcolors' color("`r(p9)'")    ||            ///
rarea l4 l5 year, `lcolors' `lcolors' color("`r(p12)'")    ||            ///
rarea l5 p_T_6 year, `lcolors' color("`r(p15)'")               ///
ytitle("Percentage of all AP exam-takers ")              ///
ylab(`yaxis', axis(2) notick) ///
xlab(${min}(2)${max}) ///
yscale(range(0 100) axis(1))      ///
yscale(range(0 100) axis(2) lstyle(none))      ///
ytitle("", axis(2))               ///
plotregion(margin(zero))          ///
aspect(1)                         ///
legend(off) ///
xline(2019, lcolor(white))
graphsout "${imgs}fig_1", type(pdf svg) replace

local cnt=1
local graph
local assort="plotregion(margin(zero)) legend(off) aspect(1)"
local yscale1=`"yscale(range(0 100)) ytitle("Percentage of AP exams with a qualifying score")"'
local yscale2=`"yscale(range(0 100) axis(2) lstyle(none)) ylabel(, notick axis(2)) ytitle("", axis(2))"'
local xlabel="xlabel(${min}(2)${max})"
local ylabel="ylabel(0(25)100)"
local ltype="lpattern(solid)"
local yaxis
colorpalette viridis, nograph select(1 3 6 9 12 15)
local colors=`"`r(p)'"'
preserve
	labvarch p_qual_score_?, after(":")
	forvalues i=0/5 {
		local axis
		if `i'==0 local axis="yaxis(1)"
		if `i'==1 local axis="yaxis(2)"
		quietly sum p_qual_score_`i' if year==${max}
		local yaxis=`"`yaxis' `r(max)' "`:variable label p_qual_score_`i''""'
		local graph=`"`graph' (line p_qual_score_`i' year, lcolor("`:word `cnt' of `colors''") `ltype' `axis')"'
		local ++cnt
	}

	twoway `graph', `yscale1' `yscale2' `assort' `xlabel' `ylabel' ///
		ylab(`yaxis', axis(2))
graphsout "${imgs}fig_2", type(pdf svg) replace
restore

local graph 
cap drop zero
cap drop l?
gen zero = 0
gen l1 = (n_T_0)
gen l2 = (n_T_0 + n_T_1)
gen l3 = (n_T_0 + n_T_1+n_T_2)
gen l4 = (n_T_0 + n_T_1+n_T_2+n_T_3)
gen l5 = (n_T_0 + n_T_1+n_T_2+n_T_3+n_T_4)
gen l6 = (n_T_0 + n_T_1+n_T_2+n_T_3+n_T_4+n_T_5)

sort year
// collect the labels for the second y-axis
local mid = l1[_N]/2
local yaxis `"`mid' "Am. Ind.""'

local mid = (l2[_N]-l1[_N])/2 + l1[_N]
local yaxis `"`yaxis' `mid' "Asian""'

local mid = (l3[_N]-l2[_N])/2 + l2[_N]
local yaxis `"`yaxis' `mid' "Hispanic""'

local mid = (l4[_N]-l3[_N])/2 + l3[_N]
local yaxis `"`yaxis' `mid' "Black""'

local mid = (l5[_N]-l4[_N])/2 + l4[_N]
local yaxis `"`yaxis' `mid' "White""'

local mid = (l6[_N]-l5[_N])/2 + l5[_N]
local yaxis `"`yaxis' `mid' "Other""'

local lcolors="lwidth(none)"
colorpalette viridis, nograph
set scheme plotplainblind_mwg
twoway rarea zero l1 year, color("`r(p1)'") `lcolors' yaxis(1) ||  ///
rarea l1 l2 year, yaxis(2)  `lcolors' color("`r(p3)'")  ||  ///
rarea l2 l3 year, `lcolors' color("`r(p6)'")  ||            ///
rarea l3 l4 year, `lcolors' color("`r(p9)'")    ||            ///
rarea l4 l5 year, `lcolors' `lcolors' color("`r(p12)'")    ||            ///
rarea l5 l6 year, `lcolors' color("`r(p15)'")               ///ytitle("Percentage of all AP test-takers")              ///
ylab(`yaxis', axis(2)) ///
ylab(0(1000000)5000000, format(%9.0fc)) ///
xlab(2002(2)2020) ///
yscale(range(0 100) axis(1))      ///
yscale(range(0 100) axis(2))      ///
ytitle("", axis(2))   ///
ytitle("Total number of AP test-takers")            ///
plotregion(margin(zero))          ///
aspect(1)                         ///
legend(off) ///
xline(2019, lcolor(white))

graphsout "${imgs}fig_3", type(pdf svg png) replace

<img src='imgs/fig_1.svg' width="400" height="400">


<img src='imgs/fig_2.svg' width="400" height="400">

<img src='imgs/fig_3.svg' width="400" height="400">
